<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/master/HTAN/Python%20Notebooks/A_Guide_to_HTAN_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Guide to HTAN Data



        Title:   A Guide to HTAN Data
        Author:  Clarisse Lau
        Created: April 2024
        Updated: June 6, 2025
        Purpose: Provide an overview of HTAN data types and volumes


# 1. Introduction & Overview
The Human Tumor Atlas Network ([HTAN](https://humantumoratlas.org/)) is a National Cancer Institute (NCI)-funded Cancer Moonshot<sup>SM</sup> initiative to construct 3-dimensional atlases of the dynamic cellular, morphological, and molecular features of human cancers as they evolve from precancerous lesions to advanced disease [[Cell April 2020](https://www.sciencedirect.com/science/article/pii/S0092867420303469)]


### 1.1 Goal

This notebook aims to provide an overview of the data accessible within HTAN, presenting at-a-glance summaries, specific statistics such as total cell counts, as well as example queries and attributes that can be used to filter for data of interest. The findings presented reflect the status of HTAN Data Release 5.1.

### 1.2 Inputs, Outputs, & Data
The originating data can be found on the [HTAN Data Portal](https://humantumoratlas.org/), and the compiled tables are on the [ISB-Cancer Gateway in the Cloud](https://isb-cgc.appspot.com/bq_meta_search/).

Each query output loads to a Data Table, an interactive display of resulting columns and rows. You are able to select the link below the table to review the [Data Table Notebook](https://colab.research.google.com/notebooks/data_table.ipynb) that gives tips on filtering and further customizing the table.

### 1.3 Notes
The queries and results in this notebook correspond to ISB-CGC HTAN Release 6.0.

To choose a different release, edit the BigQuery table names in this notebook by replacing the relevant string endings:
- `r6` with a selected numbered release, e.g., `r2`
- `r6_v2` with `r5`
- `gc` with `cds`

To get results for the most current data release, replace:
- `r6` and `r6_v2` with `current`
- `HTAN_versioned` with `HTAN`

#
### ⚠️PAUSE
Notebooks associated with ISB-CGC HTAN Releases 6.0 or earlier are based on the **HTAN Phase 1 Data Model**. Please be aware that the structure, terminology, and available data elements may differ from those in Phase 2.



# 2. Environment & Module Setup

In [1]:
# import libraries
import pandas as pd

Enable interactive table formatting:

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
from google.colab.data_table import DataTable
DataTable.max_columns = 30

# 3. Google Authentication

Running the BigQuery cells in this notebook requires a Google Cloud Project. Instructions for creating a project can be found in [Google Cloud Documentation](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). The instance needs to be authorized to bill the project for queries. For more information on getting started with ISB-CGC see [Quick Start Guide to ISB-CGC](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) and alternative authentication methods can be found in [Google Cloud Authentication Documentation](https://cloud.google.com/docs/authentication).

## 3.1 Authenticating with Google Credentials



#### Option 1. Running in Google Colab

If you are using Google Colab, run the code block below to authenticate

In [4]:
from google.colab import auth
auth.authenticate_user()

#### Option 2. Running on local machine

Alternatively, if you're running the notebook locally, take the following steps to authenticate.

1.   Run `gcloud auth application-default login` on your local machine
2.   Run the command below replacing `<path to key>` with the path to your credentials file

In [ ]:
# env GOOGLE_APPLICATION_CREDENTIALS='<path to key>'

## 3.2 Initializing the Google BigQuery client


In [1]:
# Import the Google BigQuery client
from google.cloud import bigquery

# Set the Google project that will be billed for this notebook's computations
# Replace <my-project> with your BigQuery Project ID
google_project = '<my-project>'

# Create a client to access the data within BigQuery
client = bigquery.Client(google_project)

# 4. Summary of Data in HTAN

## 4.1 Data Levels by Center

In the query below, a data file listing is first obtained using the ID Provenance table (see the [HTAN_ID_Provenance_In_BQ.ipynb](https://github.com/isb-cgc/Community-Notebooks/blob/master/HTAN/Python%20Notebooks/HTAN_ID_Provenance_In_BQ.ipynb) notebook for a tutorial on using the ID Provenance table).

HTAN imaging metadata annotations differ slightly from other assay types, as they encompass multiple imaging and spatial modalities. For example, H&E, MIBI, CODEX, etc. types are all annotated under the imaging template. In order to distinguish these assay types in our summary, we utilize the `Imaging Assay Type` attribute. As `Imaging Assay Type` is exclusively found in the `ImagingLevel2` table, we conduct a sequence of joins utilizing the parent data file lineage to classify higher-level imaging data by assay type.

In the next portions of the query we split the component name e.g. `ScRNA-seqLevel2` into separate assay type (`ScRNA-seq`)and data level (`Level2`) attributes.

Lastly, we group by assay type, data level, and HTAN center, and pivot the table to provide a concise overview of all HTAN data types by center, along with the levels available.

In [2]:
summary = client.query("""
  WITH prov AS (
    SELECT DISTINCT HTAN_Data_File_ID, entityId, Component, HTAN_Center
    FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
    WHERE Component IS NOT NULL
    AND Component NOT LIKE '%Auxiliary%'
    AND Component NOT LIKE 'OtherAssay'
  ),
  img AS(SELECT * EXCEPT(HTAN_Data_File_ID) FROM (
    SELECT HTAN_Data_File_ID,Imaging_Assay_Type,entityId
    FROM `isb-cgc-bq.HTAN_versioned.imaging_level2_metadata_r6`
    WHERE Component IS NOT NULL
    UNION ALL
    SELECT il3s.HTAN_Data_File_ID,il2.Imaging_Assay_Type,il3s.entityId
    FROM `isb-cgc-bq.HTAN_versioned.imaging_level2_metadata_r6` il2
    JOIN (SELECT * FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
      WHERE Component = 'ImagingLevel3Segmentation') il3s
    ON il2.HTAN_Data_File_ID = il3s.HTAN_Parent_Data_File_ID
    UNION ALL
    SELECT il4.HTAN_Data_File_ID,il2.Imaging_Assay_Type,il4.entityId
    FROM `isb-cgc-bq.HTAN_versioned.imaging_level2_metadata_r6` il2
    JOIN (SELECT * FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
      WHERE Component = 'ImagingLevel3Segmentation') il3s
    ON il2.HTAN_Data_File_ID = il3s.HTAN_Parent_Data_File_ID
    JOIN (SELECT * FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
      WHERE Component = 'ImagingLevel4') il4
    ON il3s.HTAN_Data_File_ID = il4.HTAN_Parent_Data_File_ID
  )
  ),
  files AS (
    SELECT HTAN_Center,array_to_string([SPLIT(Component, 'Level')[OFFSET(0)], Imaging_Assay_Type], ' - ') AS Assay,
    REGEXP_EXTRACT(Component, r'Level\d') AS Level
    FROM prov
    LEFT JOIN img USING(entityId)
    GROUP BY HTAN_Center, Assay, Level
    ORDER BY HTAN_Center, Assay, Level
  )
  SELECT HTAN_Center, Assay, STRING_AGG(DISTINCT Level ORDER BY Level) AS Levels
  FROM files
  GROUP BY HTAN_Center, Assay
""").result().to_dataframe()

In [3]:
summary.pivot(index='Assay', columns='HTAN_Center', values='Levels').fillna('')

HTAN_Center,HTAN BU,HTAN CHOP,HTAN DFCI,HTAN Duke,HTAN HMS,HTAN HTAPP,HTAN MSK,HTAN OHSU,HTAN SRRS,HTAN Stanford,HTAN TNP - TMA,HTAN TNP SARDANA,HTAN Vanderbilt,HTAN WUSTL
Assay,,,,,,,,,,,,,,
10xVisiumSpatialTranscriptomics-RNA-seq,,,,,,"Level1,Level2,Level3",,,,,,,"Level1,Level2,Level3,Level4","Level1,Level2,Level3,Level4"
BulkMethylation-seq,,,,,,,,,,"Level1,Level2",,,,
BulkRNA-seq,"Level1,Level2,Level3",,,"Level1,Level2,Level3","Level1,Level2,Level3",Level2,,"Level1,Level2,Level3",,"Level1,Level3",,,,Level1
BulkWES,Level1,"Level1,Level2,Level3",Level2,"Level1,Level2,Level3",,Level2,,"Level1,Level2,Level3",,"Level1,Level2",,,"Level1,Level2,Level3",Level1
ElectronMicroscopy,,,,,,,,"Level1,Level2",,,,,,
ExSeqMinimal,,,,,,,,,,,,,,
HI-C-seq,,,,,,,,,,"Level1,Level2,Level3",,,,
Imaging,,,,Level4,,Level4,,,,Level1,,,,
Imaging - CODEX,,"Level2,Level3,Level4",,,,Level2,,,,"Level2,Level3,Level4",,Level2,,Level2


## 4.2 Number of files per center and assay type

This next table provides a long-form breakdown of the number of data files submitted by each HTAN center per assay type and level.

In [4]:
all_files = client.query("""
  SELECT Component, HTAN_Center, Count(*) AS Count
  FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
  WHERE Component LIKE '%Level%'
  GROUP BY Component, HTAN_Center
  ORDER BY Component, HTAN_Center
  """).result().to_dataframe()

In [5]:
all_files.pivot_table(
    index='Component',
    columns='HTAN_Center',
    values='Count',
    fill_value=0).astype(int)

HTAN_Center,HTAN BU,HTAN CHOP,HTAN DFCI,HTAN Duke,HTAN HMS,HTAN HTAPP,HTAN MSK,HTAN OHSU,HTAN SRRS,HTAN Stanford,HTAN TNP - TMA,HTAN TNP SARDANA,HTAN Vanderbilt,HTAN WUSTL
Component,,,,,,,,,,,,,,
10xVisiumSpatialTranscriptomics-RNA-seqLevel1,0,0,0,0,0,400,0,0,0,0,0,0,96,304
10xVisiumSpatialTranscriptomics-RNA-seqLevel2,0,0,0,0,0,400,0,0,0,0,0,0,96,304
10xVisiumSpatialTranscriptomics-RNA-seqLevel3,0,0,0,0,0,336,0,0,0,0,0,0,192,1250
10xVisiumSpatialTranscriptomics-RNA-seqLevel4,0,0,0,0,0,0,0,0,0,0,0,0,192,792
BulkMethylation-seqLevel1,0,0,0,0,0,0,0,0,0,80,0,0,0,0
BulkMethylation-seqLevel2,0,0,0,0,0,0,0,0,0,80,0,0,0,0
BulkRNA-seqLevel1,826,0,0,885,444,0,0,88,0,420,0,0,0,434
BulkRNA-seqLevel2,317,0,0,774,444,154,0,56,0,0,0,0,0,0
BulkRNA-seqLevel3,186,0,0,1548,222,0,0,38,0,72,0,0,0,0


## 4.3 Total Cell Counts in Google BigQuery
In this section, we aim to retrieve the total cell counts across HTAN genomic and spatial assay data tables in Google BigQuery. Given the varying formats of assay tables, we adopt different approaches to gather cell count information, distinguishing between narrow and non-narrow formats.

In [6]:
# obtain assay tables containing concatenated data derived from CSVs
full = client.query("""
  SELECT table_name
  FROM `isb-cgc-bq.HTAN_versioned`.INFORMATION_SCHEMA.TABLES
  WHERE (table_name LIKE 'imaging_level4%' AND table_name NOT LIKE '%metadata%')
  OR table_name = 'scRNAseq_HTAPP_level4_r6'
  ORDER BY table_name
""").result().to_dataframe()

# obtain assay tables derived from h5ads
narrow = client.query("""
  SELECT table_name
  FROM `isb-cgc-bq.HTAN_versioned`.INFORMATION_SCHEMA.TABLES
  WHERE (table_name LIKE 'scRNAseq%' AND table_name NOT LIKE '%metadata%')
  AND (table_name NOT LIKE 'scRNAseq_VUMC%' OR table_name LIKE '%cellxgene%')
  AND table_name NOT LIKE 'scRNAseq_HTAPP_level4_%'
  ORDER BY table_name
""").result().to_dataframe()

In [7]:
def row_count(table):
  '''
  For csv/tsv derived tables, we can count one cell per row
  '''
  count = client.query(f"SELECT COUNT(*) AS count FROM `isb-cgc-bq.HTAN_versioned.{table}`")
  result = count.result()
  for row in result:
      return row['count']

In [8]:
def obs_count(table):
  '''
  Tables that were derived from h5ads have been converted into narrow format using
  obs & var indices. We can obtain cell count by selecting the max obs index
  '''
  count = client.query(f"SELECT MAX(iObs) AS count FROM `isb-cgc-bq.HTAN_versioned.{table}`")
  result = count.result()
  for row in result:
      return row['count']

In [9]:
cell_count = 0
for t in list(full['table_name']):
  cell_count = cell_count + row_count(t)
for t in list(narrow['table_name']):
  cell_count = cell_count + obs_count(t)


The total cell count in Google BigQuery (genomic and spatial data) is around 204 million cells

In [10]:
cell_count

204060254

# 5. Specific modes of access and identification

## 5.1 Identifying data by HTAN Data Release

Here we demonstrate how users can filter files based on a specific HTAN Data Release, such as `Release 6.0`. We accomplish this by again utilizing the ID Provenance table, which contains information on all HTAN data files.

In [11]:
id_prov = client.query("""
  SELECT Component, Data_Release, HTAN_Center, Count(*) AS Count
  FROM `isb-cgc-bq.HTAN_versioned.id_provenance_r6`
  WHERE Data_Release = 'Release 6.0'
  GROUP BY Component, Data_Release, HTAN_Center
""").result().to_dataframe()
id_prov

,Component,Data_Release,HTAN_Center,Count
0,BulkRNA-seqLevel2,Release 6.0,HTAN BU,317
1,BulkRNA-seqLevel3,Release 6.0,HTAN BU,186
2,ImagingLevel2,Release 6.0,HTAN DFCI,16528
3,ScRNA-seqLevel3,Release 6.0,HTAN DFCI,63
4,ScRNA-seqLevel1,Release 6.0,HTAN DFCI,20
5,BulkWESLevel2,Release 6.0,HTAN DFCI,8
6,ImagingLevel3Segmentation,Release 6.0,HTAN OHSU,437
7,ImagingLevel2,Release 6.0,HTAN OHSU,67
8,ImagingLevel4,Release 6.0,HTAN OHSU,402
9,ScATAC-seqLevel1,Release 6.0,HTAN OHSU,28


## 5.2 Identifying Data in the NIH General Commons (GC)

Similarly, we can identify data released on the GC, previously Cancer Data Service (CDS), using the `isb-cgc-bq.HTAN_versioned.gc_drs_map_r6_v2` BigQuery table. This table lists all ~60,000 data files available in the GC as of Release 6.0 (the DCC submits data to the GC following each major data release).

In [15]:
gc = client.query("""
SELECT * FROM `isb-cgc-bq.HTAN_versioned.gc_drs_map_r6_v2`
""").result().to_dataframe()

gc

,name,entityId,HTAN_Data_File_ID,drs_uri,HTAN_Center
0,1-PCGA02-20005-1005102_S1_R1_001.fastq.gz,syn53283905,HTA3_70005_3600639,dg.4DFC/20852dd0-9085-11ef-b71c-3f2710856ceb,HTAN BU
1,1-PCGA02-20005-1005102_S1_R1_001.fastq.gz,syn53283905,HTA3_70005_3600639,dg.4DFC/70d84a82-03d2-11ef-a132-7f12e8801d7c,HTAN BU
2,1-PCGA02-20005-1005102_S1_R2_001.fastq.gz,syn53283874,HTA3_70005_3221388,dg.4DFC/20a8a8be-9085-11ef-a3a5-3f109c193228,HTAN BU
3,1-PCGA02-20005-1005102_S1_R2_001.fastq.gz,syn53283874,HTA3_70005_3221388,dg.4DFC/70e79a14-03d2-11ef-9811-d7ff64fafaf4,HTAN BU
4,10-PCGA02-20152-1000048_S10_R1_001.fastq.gz,syn53283759,HTA3_70152_0185853,dg.4DFC/70e0e002-03d2-11ef-9751-8fbe5ca478f7,HTAN BU
...,...,...,...,...,...
65779,ht565b1_s1h2a3y1e1.GCTTGAGACG-ACGCAAGAAG.HH5NV...,syn61464414,HTA12_255_1008,dg.4DFC/1e0c2220-9085-11ef-bdff-8f6d3dd0a631,HTAN WUSTL
65780,ht565b1_s1h2a3y1e1.GCTTGAGACG-ACGCAAGAAG.HH5NV...,syn61464333,HTA12_255_1009,dg.4DFC/1e1bb230-9085-11ef-8243-83a15db66a4e,HTAN WUSTL
65781,mCRC_S16-32775_Met__20230826.ome.tiff,syn61463092,HTA12_279_2002,dg.4DFC/7E491980-65C8-4D2F-BAA7-A992C9B2617E,HTAN WUSTL
65782,mCRC_S16-38794_Met__20230823.ome.tiff,syn61464203,HTA12_280_2004,dg.4DFC/EECF1505-68B7-4416-857E-0BB9207C9F74,HTAN WUSTL


## 5.3 Identifying Assay Data in ISB-CGC BigQuery

The HTAN Data Coordinating Center (DCC) has now made over 1000 data files available in ISB-CGC BigQuery, including tabular assay data and channel metadata files.

These files are structured in two main formats: in some instances, each data file corresponds directly to a BigQuery table, while in others, files within a dataset are combined into a single BigQuery table.

The `isb-cgc-bq.HTAN_versioned.dataFileSynapseID_to_BigQueryTableID_map_r6` table can be utilized to identify the presence of data files in Google BigQuery using their Synapse IDs, and determine the tables they are associated with.

In [16]:
syn_bq_map = client.query("""
SELECT * FROM `isb-cgc-bq.HTAN_versioned.dataFileSynapseID_to_BigQueryTableID_map_r6`
""").result().to_dataframe()

syn_bq_map

,entityId,bq_table_id
0,syn26535445,isb-cgc-bq.HTAN.imaging_level4_OHSU_current
1,syn53275842,isb-cgc-bq.HTAN.imaging_level4_OHSU_current
2,syn53275838,isb-cgc-bq.HTAN.imaging_level4_OHSU_current
3,syn26535453,isb-cgc-bq.HTAN.imaging_level4_OHSU_current
4,syn31547329,isb-cgc-bq.HTAN.imaging_level4_OHSU_current
...,...,...
1316,syn51229375,isb-cgc-bq.seurat_scATAC_patient_clean
1317,syn51229367,isb-cgc-bq.seurat_scATAC_patient_clean
1318,syn51229373,isb-cgc-bq.seurat_scATAC_patient_clean
1319,syn51229368,isb-cgc-bq.seurat_scATAC_patient_clean


A breakdown of the number of data files contained in each BigQuery assay table

In [17]:
syn_bq_map.bq_table_id.value_counts()

bq_table_id
isb-cgc-bq.HTAN.imaging_channel_metadata_current                                                   423
isb-cgc-bq.HTAN.imaging_level4_TNP_TMA_phase1_current                                              352
isb-cgc-bq.HTAN.imaging_level4_TNP_TMA_phase3_current                                              176
isb-cgc-bq.HTAN.imaging_level4_OHSU_current                                                        120
isb-cgc-bq.HTAN.imaging_level4_HMS_orion_current                                                    75
isb-cgc-bq.HTAN.scRNAseq_HTAPP_level4_current                                                       47
isb-cgc-bq.HTAN.imaging_level4_HMS_crc_mask_current                                                 40
isb-cgc-bq.HTAN.imaging_level4_HMS_crc_current                                                      32
isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current                                                 20
isb-cgc-bq.HTAN.imaging_level4_HTAPP_merfish_current         

# 6. Relevant Citations and Links



[HTAN Portal](https://humantumoratlas.org/)

[Overview paper, Cell, April 2020](https://www.sciencedirect.com/science/article/pii/S0092867420303469)

